In [2]:
import numpy as np
import scipy.integrate
import torch
import math
from scipy.optimize import fsolve
from sympy import symbols, lambdify, I, latex
from scipy.integrate import solve_ivp
from torch.utils.data import DataLoader, TensorDataset

In [5]:
# Define all the variables
x = symbols("x")
x_bar = symbols(r"\bar{x}")
omega_0 = symbols(r"\omega_0", real = True)

# Define Hamilton's equations
def h_2n(n_val, x_val, x_bar_val, omega_0_val):
	return ((-1)**(n_val+1) * omega_0_val**2 / math.factorial(2*n_val)) * (1 / (2*omega_0_val))**n_val * (x_val + x_bar_val)**(2*n_val)

# Define the Hamiltonian function: h 
h = omega_0 * x * x_bar

for i in range(2,4):
	h += h_2n(i, x, x_bar, omega_0)

# Differentiate with respect to x_bar
dhdx_bar = lambdify([x, x_bar, omega_0], h.diff(x_bar) * I)

def hamiltonian_system(omega):
    def diff_func(time, z):
        x_complex = z[0] + 1j * z[1]  
        dxdt = dhdx_bar(x_complex, x_complex.real - 1j * x_complex.imag, omega)
        return [dxdt.real, dxdt.imag]
    return diff_func

def solve_for_x(H, omega_0_val):
    equation = lambdify(x, h.subs({x_bar: x, omega_0: omega_0_val}) - H)
    x0_guess = np.sqrt(H / omega_0_val)
    x_solution = fsolve(equation, x0_guess)
    return x_solution[0]

def Hamiltonian_encoder_gen(init_H, init_omega, t_span, t_steps):
	t_eval = np.linspace(*t_span, t_steps)

	# Set initial condition for x and x̄ based on chosen 
	# energy H as sqrt(H/omega_0) with phase as 0
	x0 = solve_for_x(init_H, init_omega)
	# x0 = fsolve(lambda x: h_f(x + 0j, 0 + 0j, 1).real - H, [2])
	z0 = [x0.real, x0.imag]

	diff_func = hamiltonian_system(init_omega)

	# Solve the ODEs
	sol = solve_ivp(diff_func, t_span, z0, t_eval=t_eval, rtol=1e-8, atol=1e-8)
	x_sol = sol.y[0] + 1j * sol.y[1]

	# Compute action variable J and action angle phi
	J_sol = np.abs(x_sol) ** 2
	phi_sol = np.angle(x_sol)

	return x_sol, J_sol, phi_sol

In [11]:
x, J, phi = Hamiltonian_encoder_gen(2, 1, (1, 10), 1000)

SympifyError: SympifyError: array([ 1.94575389e+00+0.j        ,  1.94573685e+00+0.00378279j,
        1.94568573e+00+0.00756569j,  1.94560053e+00+0.01134885j,
        1.94548125e+00+0.01513237j,  1.94532788e+00+0.01891637j,
        1.94514041e+00+0.02270099j,  1.94491885e+00+0.02648634j,
        1.94466318e+00+0.03027254j,  1.94437339e+00+0.03405972j,
        1.94404949e+00+0.03784801j,  1.94369144e+00+0.04163752j,
        1.94329926e+00+0.04542837j,  1.94287291e+00+0.04922071j,
        1.94241239e+00+0.05301463j,  1.94191769e+00+0.05681029j,
        1.94138878e+00+0.06060779j,  1.94082565e+00+0.06440726j,
        1.94022828e+00+0.06820884j,  1.93959665e+00+0.07201265j,
        1.93893075e+00+0.07581882j,  1.93823054e+00+0.07962747j,
        1.93749601e+00+0.08343873j,  1.93672713e+00+0.08725273j,
        1.93592388e+00+0.09106962j,  1.93508623e+00+0.0948895j ,
        1.93421415e+00+0.09871253j,  1.93330762e+00+0.10253883j,
        1.93236660e+00+0.10636853j,  1.93139106e+00+0.11020178